In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage
# from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.node_parser import SentenceSplitter 
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
import chromadb
import pandas as pd 
import openai
import os
import getpass

In [2]:
data_path = os.path.join('/workspace/data/')
index_path = os.path.join('/workspace/db/local')

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [6]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [5]:
desc_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'desc'))
features_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'features'))
qualification_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'qualification'))

In [7]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [8]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [9]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
features_idx = load_index_from_storage(features_storage_context, index_id='loan_tmp', service_context=service_context)
desc_idx = load_indices_from_storage(desc_storage_context, index_ids=['card_tmp', 'loan_tmp', 'deposit_tmp'], service_context=service_context)
qualification_idx = load_index_from_storage(qualification_storage_context, index_id='loan_tmp', service_context=service_context)

In [10]:
query_engine = desc_idx[0].as_query_engine()
query_engine.query('보증 자격')

Response(response='Context information is below.\n---------------------\n서민금융진흥원의 보증을 통해 신청 가능한 카드\n\n군인연금수급자임을 증명하는 신분증 기능에 전국 군복지시설 및 5대 대표업종 청구할인, 버스요금 지원서비스\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: 보증 자격\nAnswer: ', source_nodes=[NodeWithScore(node=TextNode(id_='902300ff-cbca-47f7-bd8d-5235a4c79b5c', embedding=None, metadata={'category': 'card'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='카드_1016', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'card'}, hash='f1d7cec412088a7b70fb8e6387a4e2eb1e735924b3add09139ec520f6b849c1f'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='5cd004e9-c047-437d-a69c-60ae0e78bff8', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'card'}, hash='07f50829d05259589f87b15fbb2aa65512a9f817d23e35896e89eed6b02d4a86'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(n

### Retrieve 

In [11]:
from llama_index.retrievers import VectorIndexRetriever 
from llama_index.query_engine import RetrieverQueryEngine 
from llama_index.postprocessor import SimilarityPostprocessor 

In [12]:
retriever = VectorIndexRetriever(
    index = features_idx,
    service_context=service_context,
    similarity_top_k = 10, 
    verbose=True
)

In [13]:
from llama_index import QueryBundle 

nodes = retriever.retrieve(QueryBundle('보증 대출'))

In [14]:
print(f'num of nodes: {len(nodes)}')
nodes[1].text, nodes[1].score

num of nodes: 10


('국민행복기금으로부터 신용보증서를 발급 받은 고객의 연20%이상의 고금리대출을 은행의 저금리 대출로 전환해주는 대출 상품',
 0.721333835387794)

In [15]:
nodes[0].node.relationships

{<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='대출_852', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'loan'}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba'),
 <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='95435469-e1e8-428f-ade6-2ee2101f5924', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'loan'}, hash='efbbf86cada436a84c1203c58317827e0ca887812e09d5beb05c33accc12d6f0'),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5ffb1ab1-6099-41f3-8b92-c0eb66f1ad78', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='556a6407d88aebf899ab1c287cc6db398c447d5032a4d4a23f905fc75c6efb68')}

In [16]:
nodes[0]

NodeWithScore(node=TextNode(id_='5a8b8c76-d97f-4ee7-87ee-aeb3e60cdacd', embedding=None, metadata={'category': 'loan'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='대출_852', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'loan'}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='95435469-e1e8-428f-ade6-2ee2101f5924', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'loan'}, hash='efbbf86cada436a84c1203c58317827e0ca887812e09d5beb05c33accc12d6f0'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5ffb1ab1-6099-41f3-8b92-c0eb66f1ad78', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='556a6407d88aebf899ab1c287cc6db398c447d5032a4d4a23f905fc75c6efb68')}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba', text='제2금융권 전세자금대출을 전액 상환하기 위해 주택금융신용보증서를 담보로 한 대출', star

In [49]:
nodes[0].node.relationships[[key for key in nodes[0].node.relationships.keys()][0]].node_id

'대출_852'

In [44]:
nodes[0].node.relationships.items()

dict_items([(<NodeRelationship.SOURCE: '1'>, RelatedNodeInfo(node_id='대출_852', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'loan'}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba')), (<NodeRelationship.PREVIOUS: '2'>, RelatedNodeInfo(node_id='95435469-e1e8-428f-ade6-2ee2101f5924', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'loan'}, hash='efbbf86cada436a84c1203c58317827e0ca887812e09d5beb05c33accc12d6f0')), (<NodeRelationship.NEXT: '3'>, RelatedNodeInfo(node_id='5ffb1ab1-6099-41f3-8b92-c0eb66f1ad78', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='556a6407d88aebf899ab1c287cc6db398c447d5032a4d4a23f905fc75c6efb68'))])

In [20]:
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage

desc_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'desc'))
features_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'features'))
qualification_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'qualification'))

In [25]:
from llama_index.storage.docstore import SimpleDocumentStore

desc_docstore = SimpleDocumentStore.from_persist_path(os.path.join(index_path, 'desc', 'docstore.json'))
features_docstore = SimpleDocumentStore.from_persist_path(os.path.join(index_path, 'features', 'docstore.json'))
qualification_docstore = SimpleDocumentStore.from_persist_path(os.path.join(index_path, 'qualification', 'docstore.json'))

In [51]:
from llama_index.vector_stores import SimpleVectorStore

desc_vectorstore = SimpleVectorStore.from_persist_path(os.path.join(index_path, 'desc', 'default__vector_store.json'))
features_vectorstore = SimpleVectorStore.from_persist_path(os.path.join(index_path, 'features', 'default__vector_store.json'))
qualification_vectorstore = SimpleVectorStore.from_persist_path(os.path.join(index_path, 'qualification', 'default__vector_store.json'))

In [33]:
features_docstore.get_node(nodes[0].node_id)

TextNode(id_='5a8b8c76-d97f-4ee7-87ee-aeb3e60cdacd', embedding=None, metadata={'category': 'loan'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='대출_852', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'loan'}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='95435469-e1e8-428f-ade6-2ee2101f5924', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'loan'}, hash='efbbf86cada436a84c1203c58317827e0ca887812e09d5beb05c33accc12d6f0'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5ffb1ab1-6099-41f3-8b92-c0eb66f1ad78', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='556a6407d88aebf899ab1c287cc6db398c447d5032a4d4a23f905fc75c6efb68')}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba', text='제2금융권 전세자금대출을 전액 상환하기 위해 주택금융신용보증서를 담보로 한 대출', start_char_idx=0, end_c

In [35]:
print(features_docstore.get_ref_doc_info(nodes[0].node_id))

None


In [60]:
desc_dict = desc_vectorstore.to_dict()
features_dict = features_vectorstore.to_dict()
qualification_dict = qualification_vectorstore.to_dict()

In [55]:
desc_dict.keys()

dict_keys(['embedding_dict', 'text_id_to_ref_doc_id', 'metadata_dict'])

In [63]:
features_dict['metadata_dict'][nodes[0].node_id]['document_id']

'대출_852'

### PostProcessor 

In [ ]:
from llama_index.postprocessor import KeywordNodePostprocessor 
from llama_index.postprocessor import SimilarityPostprocessor, CohereRerank
from llama_index.schema import Node, NodeWithScore 

In [ ]:
node_postprocessors = [
    SimilarityPostprocessor(similarity_cutoff=0.71),
    KeywordNodePostprocessor(required_keywords=['대출'])
]

In [ ]:
# similarity post processor 
processor = SimilarityPostprocessor(similarity_cutoff=0.71)
filtered = processor.postprocess_nodes(nodes)

In [ ]:
postprocessor = KeywordNodePostprocessor(
    required_keywords=['대출']
)

len(postprocessor.postprocess_nodes(filtered))

In [ ]:
filtered[0]

In [ ]:
len(filtered)

In [ ]:
filtered[0].score, filtered[0].text

In [ ]:
filtered[0].node_id  # node_id == id_, 해당 정보는 default_vector_store  -> text_id_to_ref_doc_id로 매핑가능 

In [ ]:
from llama_index.postprocessor import SentenceEmbeddingOptimizer 

postprocessor = SentenceEmbeddingOptimizer(percentile_cutoff=0.5)

#### Re-ranking 

##### Sentence Transformers 

In [ ]:
from llama_index.postprocessor import SentenceTransformerRerank 

rerank = SentenceTransformerRerank(
    model='bongsoo/albert-small-kor-cross-encoder-v1',
    top_n=3
)

In [ ]:
query_engine = desc_idx[0].as_query_engine(node_postprocessor=[rerank])
query_engine.query('보증 자격')

In [ ]:
query_engine = desc_idx[0].as_query_engine(node_postprocessor=[rerank])
nodes = query_engine.query('서민들을 위한 카드 추천해줘')

In [ ]:
print(nodes.response)

##### Cohere

In [ ]:
import cohere 

In [ ]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")
co_api = os.environ["COHERE_API_KEY"]

In [ ]:
cohere_rerank = CohereRerank(api_key=co_api, top_n=2) 

In [ ]:
query_engine = desc_idx[0].as_query_engine(node_postprocessor=[cohere_rerank])
query_engine.query('보증 자격') 

### Retrieve, PostProcess  -> Return Node 

In [ ]:
def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, service_context=None, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index = features_idx,
        service_context=service_context,
        similarity_top_k = 10, 
        verbose=True
    )
    retrieved_nodes = retriever.retrieve(query_bundle)
    print(len(retrieved_nodes), retrieved_nodes[0].score)
    node_postprocessors = SimilarityPostprocessor(similarity_cutoff=0.64)
    processed_nodes = node_postprocessors.postprocess_nodes(retrieved_nodes)
    print(len(processed_nodes))
    if with_reranker:
        # configure reranker
        reranker = SentenceTransformerRerank(
            model='bongsoo/albert-small-kor-cross-encoder-v1',
            top_n=reranker_top_n,
        )
        reranked_nodes = reranker.postprocess_nodes(
            processed_nodes, query_bundle
        )
    return reranked_nodes

In [ ]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [ ]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
features_idx = load_index_from_storage(features_storage_context, index_id='loan_tmp', service_context=service_context)
desc_idx = load_indices_from_storage(desc_storage_context, index_ids=['card_tmp', 'loan_tmp', 'deposit_tmp'], service_context=service_context)
qualification_idx = load_index_from_storage(qualification_storage_context, index_id='loan_tmp', service_context=service_context)

In [ ]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

In [ ]:
with_reranker = True

In [ ]:
query = '담보대출 관련 대출 상품 정보 알려줘'

In [ ]:
get_retrieved_nodes(query, service_context=service_context, with_reranker=with_reranker)[0]

In [ ]:
get_retrieved_nodes(query, service_context=service_context, with_reranker=with_reranker)[0].text

### llm config test 

In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
     tokenizer_name="davidkim205/komt-mistral-7b-v1",
    model_name="davidkim205/komt-mistral-7b-v1",
    device_map="cuda",
    # offload_folder='offload',
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)

In [ ]:
mistral_service_context = ServiceContext.from_defaults(llm=llm)

In [ ]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
features_idx = load_index_from_storage(features_storage_context, index_id='loan_tmp', service_context=service_context)
desc_idx = load_indices_from_storage(desc_storage_context, index_ids=['card_tmp', 'loan_tmp', 'deposit_tmp'], service_context=service_context)
qualification_idx = load_index_from_storage(qualification_storage_context, index_id='loan_tmp', service_context=service_context)

In [ ]:
from llama_index.llms import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "사용자 쿼리와 유사한 문맥 정보는 다음과 같아. \n"
            "---------------------\n"
            "{context_str}"
            "\n---------------------\n"
            "주어진 문맥 정보를 바탕으로, 사용자 질문에 대답해줘: {query_str}\n"
            "대답은 한국어로 하고, 주어진 문맥이 별로 도움되지 않는다면, 너가 대답을 만들어줘"
        ),
    ),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [ ]:
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "우리는 기존 대답을 정제할 기회가 있어"
            "(꼭 필요한 경우에만) 다음 추가 문맥 정보를 사용해서.\n"
            "------------\n"
            "{context_msg}\n"
            "------------\n"
            "새로운 문맥 정보가 주어졌을 때, 기존 대답을 보다 낫게 정제해봐"
            "대답해야 할 사용자 발화는 이거야: {query_str}. "
            "추가로 주어진 문맥 정보가 유용하지 않다면, 기존 대답을 출력해줘.\n"
            "기존 대답: {existing_answer}"
        ),
    ),
]

refine_template = ChatPromptTemplate(chat_refine_msgs)

In [ ]:
response = features_idx.as_query_engine(
    service_context=mistral_service_context, text_qa_template=text_qa_template, refine_template=refine_template
).query(query)